# 1. Setup

## 1.1. Installing and importing libraries to the collab new session (therefore new environment)

In [1]:
!python -m nltk.downloader 'punkt'
#restart the runtime once installed

/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
!pip install "tqdm==4.43.0"

     |████████████████████████████████| 59 kB 341 kB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [3]:
!pip install sumy

     |████████████████████████████████| 83 kB 1.7 MB/s 
     |████████████████████████████████| 10.1 MB 14.5 MB/s 
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21711 sha256=40197a653d9e3671f264aad9fbbcc3eccc4fe7c6546506b52a77535db1fdabd9
  Stored in directory: /root/.cache/pip/wheels/d4/bf/51/81d27ad638e1a6dca4f362ecc33d1e2c764b8ea7ec751b8fc1
  Created wheel for pycountry: filename=pycountry-20.7.3-py2.py3-none-any.whl size=10746883 sha256=cb328e5376f4e49287d7ff6020708c8c3467a44e82d163c1db7522dfbf9f6a5c
  Stored in directory: /root/.cache/pip/wheels/57/e8/3f/120ccc1ff7541c108bc5d656e2a14c39da0d824653b62284c6
Successfully built breadability pycountry


In [4]:
!pip install datasets

     |████████████████████████████████| 542 kB 9.7 MB/s 
     |████████████████████████████████| 243 kB 55.3 MB/s 
     |████████████████████████████████| 43 kB 1.2 MB/s 
     |████████████████████████████████| 118 kB 48.0 MB/s 


In [5]:
!pip install rouge_score

In [6]:
# Importing necessary librairies
import datasets
from datasets import load_metric
import nltk
import numpy as np
import pandas as pd
import pickle 
from sklearn.model_selection import train_test_split
import spacy
from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.summarizers.sum_basic import SumBasicSummarizer
import tqdm
from google.colab import drive

## 1.2. Importing personal drive to file in order to load the data


In [7]:
# Mount Google Drive to this Notebook instance.
drive.mount('/content/drive')

Mounted at /content/drive


## 2. Data Formatting

## 2.1. Loading clean data 

In [8]:
# We load the cleaned data from a pickle format
file ='/content/drive/MyDrive/Colab Notebooks/Data/final_clean.pickle'
df = pd.read_pickle(file)
# Looking at the data
df

,description_narrative,TLDR,text_clean,text_embedding,summary_clean,summary_embedding,labels,labels_idx_list,doc_label
0,The training improved women’s knowledge on the...,The training improved women’s knowledge on the...,[The training improved women’s knowledge on th...,"[[0.76747984, -0.1894493, 0.51285785, -0.02116...",[The training improved women’s knowledge on th...,"[[0.76747984, -0.1894493, 0.51285785, -0.02116...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[0],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Illegal oil refining in the Niger Delta is inc...,CEHRD in an effort to create awareness on the ...,[Illegal oil refining in the Niger Delta is in...,"[[-0.26888534, -0.6092957, -0.23817927, -1.167...",[CEHRD in an effort to create awareness on the...,"[[-0.43648192, -0.071132354, -0.18081759, -0.4...","[0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2, 3]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,CEHRD successfully set-up 4 formal and 4 infor...,CEHRD set-up the environmental clubs with the ...,[CEHRD successfully set-up 4 formal and 4 info...,"[[-0.48964322, -1.2085572, 1.0374501, 0.021369...",[CEHRD set-up the environmental clubs with the...,"[[-0.15335692, -0.29430717, 0.58692193, -1.126...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[10],"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]"
3,PROJECT NAME : Delivering Accelerated Family P...,Delivering Accelerated Family Planning in Paki...,"[PROJECT NAME :, Delivering Accelerated Family...","[[-0.23698506, 0.15983863, -0.071198896, -1.20...",[Delivering Accelerated Family Planning in Pak...,"[[-0.50266284, -1.2923895, 0.4206834, -1.25673...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[1],"[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ..."
4,Road traffic injuries are world's eighth leadi...,The Cardiff Trauma Pack Research and Develop...,[Road traffic injuries are world's eighth lead...,"[[-0.4666822, -1.19177, 0.9945381, -0.95759714...",[ The Cardiff Trauma Pack Research and Develo...,"[[-0.4679708, -0.43502182, 0.8388376, -1.47547...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, ...","[4, 5]","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ..."
...,...,...,...,...,...,...,...,...,...
2980,Metta has been responding to the humanitarian ...,"ProjectGoal: To improve the condition of 2,854...",[Metta has been responding to the humanitarian...,"[[-0.6875755, -1.0828758, 0.32236806, -1.33987...","[ProjectGoal:, To improve the condition of 2,8...","[[-0.58209735, 0.027911462, 0.29285246, -1.098...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, ...","[7, 8]","[2980, 2980, 2980, 2980, 2980, 2980, 2980, 298..."
2981,"Destined Women is local not for profit, non-re...",GOAL: Contribute towards changing the socioeco...,"[Destined Women is local not for profit, non-r...","[[-0.3787009, -0.7928037, 0.16012278, -0.16681...","[GOAL:, Contribute towards changing the socioe...","[[-0.2746246, 0.04018628, 0.4832729, -0.996766...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[1, 7]","[2981, 2981, 2981, 2981, 2981, 2981, 2981, 298..."
2982,The project will empower secondary school stud...,The project will empower secondary school stud...,[The project will empower secondary school stu...,"[[0.3152278, -0.62204534, 1.0845532, -0.392865...",[The project will empower secondary school stu...,"[[0.3152278, -0.62204534, 1.0845532, -0.392865...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",[0],"[2982, 2982, 2982, 2982, 2982, 2982, 2982, 2982]"
2983,The project has been specifically designed to ...,The project has been specifically designed to ...,[The project has been specifically designed to...,"[[0.2448853, -0.043037124, 0.5578783, -0.51006...",[The project has been specifically designed to...,"[[0.2448853, -0.043037124, 0.5578783, -0.51006...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",[0],"[2983, 2983, 2983, 2983, 2983, 2983]"


In [9]:
# Renaming the columns and
df = pd.DataFrame({'document': df['description_narrative'],
                   'summary': df['TLDR'],
                   'label' :  df['labels']
                   })

## 2.2. Splitting in training, validation and test datasets

In [10]:
# Using skicit learn library, we split the data, keeping 80% for training, 10% for validation and 10% for test data
# we note that in this notebook we will only use the test dataset as this is a baseline model 
train_dataset, test_dataset = train_test_split(df, test_size=0.2)
validation_dataset, test_dataset = train_test_split(test_dataset, test_size=0.5)

# 3. Loading the summariser 

In [11]:
# Defining a function to parse, tokenize and summarise the text 
def summariser(doc):
  parser = PlaintextParser.from_string(doc, Tokenizer('english'))
  sumbasic = SumBasicSummarizer()
  summary = sumbasic(parser.document, 2)
  return summary

# Applying the function to the dataset
#creating an empty list
sum_basic_summaries = []

#looping in the texts of teh datasets
for doc in test_dataset['document']:
  #applying the function to each text 
  #appending the result to the list 
  sum_basic_summaries.append(summariser(doc))

# Creating a column filled with the generated summaries 
test_dataset['predicted_summaries'] = sum_basic_summaries
test_dataset = test_dataset.reset_index()
test_dataset

,index,document,summary,label,predicted_summaries
0,1011,The objective of the Support to Uruguayan Publ...,The objective of the Support to Uruguayan Publ...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",(The second component is the strengthening of ...
1,1513,Afghanistan - Tuberculosis - United Nations De...,This program has the following goals: To reduc...,"[0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",(To detect and treat at least 80% of estimated...
2,387,The objective of the Private Investment and Di...,The objective of the Private Investment and Di...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",(Introducing sustainable and resilient service...
3,1473,The objective of the Second Rural Electrificat...,The objective of the Second Rural Electrificat...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",(SHS and micro grids will be available to rura...
4,2279,Kebetkache works in four principal thematic ar...,Kebetkache works in four principal thematic ar...,"[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",(Kebetkache works in four principal thematic a...
...,...,...,...,...,...
282,1951,The development objective of the Employer Driv...,The development objective of the Employer Driv...,"[1.0, 0.0, 0.0, 0.0, 0.0]",(The development objective of the Employer Dri...
283,2950,"SIDA GROW programme, Oxfam is implementing wit...","SIDA GROW programme, Oxfam is implementing wit...","[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",(Advocating and supporting the government to f...
284,2340,ContextnPakistan is the world#s sixth most den...,"The project aims to ensure that by 2021, over...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",(More than 132.9 million people (above 60%) ar...
285,835,The development objective of the Alliance for ...,The development objective of the Alliance for ...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",(The development objective of the Alliance for...


# 4. Evaluation

## 4.1. Data Preparation

In [12]:
# Creating lists of the reference and the predicted summaries
reference_summaries = test_dataset['summary']
predicted_summaries = test_dataset['predicted_summaries']

# Converting the predicted summaries to strings 
predicted_summaries = [str(sent) for sent in predicted_summaries]

# Defining a function to clean the predicted sentences from the added format generated by text_rank
def cleaner(sent):
  sent = sent.replace("(<Sentence: ", "")
  sent = sent.replace(" <Sentence:", "")
  sent = sent.replace ('>)', "")
  sent = sent.replace('>,', "")
  return sent 

# Applying the function to both the predicted and the reference summaries for consistency
predicted_summaries = [cleaner(sent) for sent in predicted_summaries]
reference_summaries = [cleaner(sent) for sent in reference_summaries]

## 4.2. Computing the ROUGE scores

In [13]:
# Loading the ROUGE metric from Hugginface 
metric = load_metric('rouge')
rouge_scores = metric.compute(predictions=predicted_summaries, references=reference_summaries)
rouge_scores

{'rouge1': AggregateScore(low=Score(precision=0.318986467166671, recall=0.4965910121353671, fmeasure=0.36087930268920604), mid=Score(precision=0.3409886305573908, recall=0.5304623291299231, fmeasure=0.38458894112608194), high=Score(precision=0.3631553780989883, recall=0.564533913799293, fmeasure=0.40851006464289835)),
 'rouge2': AggregateScore(low=Score(precision=0.15410137095228574, recall=0.2723028999504411, fmeasure=0.18400951766731932), mid=Score(precision=0.18024392553340818, recall=0.3199142564646632, fmeasure=0.21532625959182078), high=Score(precision=0.20938123463678238, recall=0.3666413682906563, fmeasure=0.24943216834012305)),
 'rougeL': AggregateScore(low=Score(precision=0.2489274474898028, recall=0.3988318096091518, fmeasure=0.28421666112263533), mid=Score(precision=0.2703964731035311, recall=0.43631919708310635, fmeasure=0.30970262383391334), high=Score(precision=0.2955934855166963, recall=0.4774548468268184, fmeasure=0.3380839009925922)),
 'rougeLsum': AggregateScore(low=

## 4.3. View a few examples

In [14]:
print('This is the reference summary: ')
print(reference_summaries[16])
print( '   ')
print('This is the predicted summary: ')
print(predicted_summaries[16])

This is the reference summary: 
CEHRD in an effort to create awareness on the dangers of illegal refining of crude oil, developed a sustainable framework to sensitize communities in Rivers and Bayelsa states on the need to stop indulging in the act for their safety. The framework involved media advocacy strategies, targeting stakeholders at different levels and local communities involved in the act and affected by it. 
   
This is the predicted summary: 
The impacts have affected oil bearing and producing communities in the Niger Delta leading to social unrest in the region. A total of 6 talk-shows were aired each in Bayelsa and Rivers states on 93.1 Peoples FM in Yenagoa and 93.7 Rhythm FM Port Harcourt.


In [15]:
print('This is the reference summary: ')
print(reference_summaries[69])
print( '   ')
print('This is the predicted summary: ')
print(predicted_summaries[69])

This is the reference summary: 
The development objective of Rural Mobility and Connectivity Project for Guinea is to improve and sustain the rural population's road access to markets and basic services.
   
This is the predicted summary: 
The development objective of Rural Mobility and Connectivity Project for Guinea is to improve and sustain the rural population's road access to markets and basic services. It has the following three subcomponents: (i) Rural Roads Rehabilitation Preparatory and Technical Studies; (ii) Selected Rural Roads Rehabilitation Works; and (iii) Small Community facilities.


In [16]:
print('This is the reference summary: ')
print(reference_summaries[103])
print( '   ')
print('This is the predicted summary: ')
print(predicted_summaries[103])

This is the reference summary: 
Population Services International (PSI) seeks to break down barriers to access and use of voluntary, modern contraception among girls and young women (age 15-24), with the aim of reducing unintended pregnancies, reducing unsafe abortions and preventing transmission of sexually transmitted infections (STIs). 
   
This is the predicted summary: 
The Ecosystem development approach, implemented by Triggerise in India and Kenya, consists in improving health the livelihood of micro-entrepreneurs who are incentivized to improve access to contraceptive products and services. The project will deliver on 5 outcomes: 1: Increased informed demand for SRH services by girls and young women, 2: Increased access to a wide range of SRH products and services for girls and young women, 3: Improved enabling environment for youth and young women for improving their SRH rights, 4: Create wealth (via improved depth and reach of the ecosystems) and 5: Improve lives (via improve